<a href="https://colab.research.google.com/github/Akshat13shah/STT-_Ai_Assignment7/blob/main/Model_Checkpointing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group 18

1.  Paras Prashant Shirvale
* Roll Number: 23110232

2. Akshat Shah
* 23110232

GitHub Repository: https://github.com/Akshat13shah/STT-_Ai_Assignment7

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load SST2 Train dataset
train_url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv"
train_df = pd.read_csv(train_url, sep="\t")  # Tab-separated file

# Load SST2 Test dataset
test_url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/test.tsv"
test_df = pd.read_csv(test_url, sep="\t")  # Tab-separated file

# Split training data into training and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Load IMDB dataset
imdb_url = "https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv"
imdb_df = pd.read_csv(imdb_url)  # CSV file

print("Training data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_df))

Training data size: 5535
Validation data size: 1384
Test data size: 1820


In [51]:
# Assign column names to SST2 datasets if missing
train_data.columns = ['text', 'label']
val_data.columns = ['text', 'label']
test_df.columns = ['text', 'label']

# Ensure labels are integers
train_data['label'] = train_data['label'].astype(int)
test_df['label'] = test_df['label'].astype(int)

In [52]:
# Check IMDB dataset columns
print(imdb_df.columns)  # Should contain 'review' and 'sentiment'

# Convert sentiment labels to numeric (0 = negative, 1 = positive)
imdb_df['sentiment'] = imdb_df['sentiment'].map({'negative': 0, 'positive': 1})

Index(['review', 'sentiment'], dtype='object')


In [53]:
print("Missing values in SST2 Train:", train_data.isnull().sum())
print("Missing values in SST2 Test:", test_df.isnull().sum())
print("Missing values in IMDB:", imdb_df.isnull().sum())

Missing values in SST2 Train: text     0
label    0
dtype: int64
Missing values in SST2 Test: text     0
label    0
dtype: int64
Missing values in IMDB: review           0
sentiment    50000
dtype: int64


In [54]:
print("SST2 Train Label Distribution:\n", train_data['label'].value_counts())
print("SST2 Test Label Distribution:\n", test_df['label'].value_counts())
print("IMDB Sentiment Distribution:\n", imdb_df['sentiment'].value_counts())

SST2 Train Label Distribution:
 label
1    2878
0    2657
Name: count, dtype: int64
SST2 Test Label Distribution:
 label
0    911
1    909
Name: count, dtype: int64
IMDB Sentiment Distribution:
 Series([], Name: count, dtype: int64)


In [55]:
train_data.head()

,text,label
26,too bad,0
3484,rife with nutty cliches and far too much dialogue,0
63,admirers of director abel ferrara may be relie...,1
1044,"the enjoyable undercover brother , a zany mix ...",1
2594,knockaround guys plays like a student film by ...,0


In [56]:
test_df.head()

,text,label
0,"a gob of drivel so sickly sweet , even the eag...",0
1,"gangs of new york is an unapologetic mess , wh...",0
2,"we never really feel involved with the story ,...",0
3,this is one of polanski 's best films,1
4,take care of my cat offers a refreshingly diff...,1


In [57]:
imdb_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,NaN
1,A wonderful little production. <br /><br />The...,NaN
2,I thought this was a wonderful way to spend ti...,NaN
3,Basically there's a family where a little boy ...,NaN
4,"Petter Mattei's ""Love in the Time of Money"" is...",NaN


In [58]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.3):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Dropout(dropout_rate))

        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout_rate))

        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Initialize the model
input_size = 10000
hidden_sizes = [512, 256, 128, 64]
output_size = 2
model = MLP(input_size, hidden_sizes, output_size)

# Count trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters:", total_params)
print(model)

Total trainable parameters: 5293122
MLP(
  (layers): ModuleList(
    (0): Linear(in_features=10000, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [59]:
from torchsummary import summary
summary(model, (1,10000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 512]       5,120,512
              ReLU-2               [-1, 1, 512]               0
           Dropout-3               [-1, 1, 512]               0
            Linear-4               [-1, 1, 256]         131,328
              ReLU-5               [-1, 1, 256]               0
           Dropout-6               [-1, 1, 256]               0
            Linear-7               [-1, 1, 128]          32,896
              ReLU-8               [-1, 1, 128]               0
           Dropout-9               [-1, 1, 128]               0
           Linear-10                [-1, 1, 64]           8,256
             ReLU-11                [-1, 1, 64]               0
          Dropout-12                [-1, 1, 64]               0
           Linear-13                 [-1, 1, 2]             130
Total params: 5,293,122
Trainable param

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_features=10000)

# Fit and transform the training data
X_train = vectorizer.fit_transform(train_data['text'])  # Replace 'text' with the column name containing text
X_val = vectorizer.transform(val_data['text'])
X_test = vectorizer.transform(test_df['text'])

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_val = torch.tensor(X_val.toarray(), dtype=torch.float32)
X_test = torch.tensor(X_test.toarray(), dtype=torch.float32)

# Labels (assuming binary classification)
y_train = torch.tensor(train_data['label'].values, dtype=torch.long)
y_val = torch.tensor(val_data['label'].values, dtype=torch.long)
y_test = torch.tensor(test_df['label'].values, dtype=torch.long)